In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
# import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import np_utils
# from keras.models import Sequential
from tensorflow.keras.models import Sequential
# 
from keras import optimizers
# from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
# from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2D,  BatchNormalization
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
DEFAULT_IMAGE_SIZE = tuple((224, 224))

# Number of images used to train the model
N_IMAGES = 9658

# Path to the dataset folder
root_dir = 'C:\\Users\\cnet\\Downloads\\For_MRI\\dataset\\mri_identify'

train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

print("333333")


333333


In [3]:
from keras.preprocessing.image import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None



In [4]:
from os import listdir
    
image_list, label_list = [], []
    
try:
    print("[INFO] Loading images ...")
    brain_disease_folder_list = listdir(train_dir)

    for brain_disease_folder in brain_disease_folder_list:
        print(f"[INFO] Processing {brain_disease_folder} ...")
        brain_disease_image_list = listdir(f"{train_dir}/{brain_disease_folder}/")

        for image in brain_disease_image_list[:N_IMAGES]:
            image_directory = f"{train_dir}/{brain_disease_folder}/{image}"
            if image_directory.endswith(".png")==True or image_directory.endswith(".jpg")==True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(brain_disease_folder)

    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

np_image_list = np.array(image_list, dtype=np.float16) / 225.0

[INFO] Loading images ...
[INFO] Processing mri ...
[INFO] Processing non_mri ...
[INFO] Image loading completed


In [5]:
import pickle
image_len = len(image_list)
print(f"Total number of images: {image_len}")

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

pickle.dump(label_binarizer,open('brain_disease_label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

print("Total number of classes: ", n_classes)



Total number of images: 9351
Total number of classes:  2


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
# checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

print("[INFO] Splitting data to train and test...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 


EPOCHS = 10
STEPS = 292
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 224
HEIGHT = 224
DEPTH = 3

from tensorflow.keras.layers import Input, ZeroPadding2D
from tensorflow.keras.models import Model


# Number of classes
num_classes = 2  # assuming 4 classes: glioma, meningioma, notumor, pituitary

# # Perform one-hot encoding on the labels
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

input_shape = (224, 224, 3)
# num_classes = 4



model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flattening layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_classes, activation='softmax'))  # Assuming num_classes is the number of classes for tumor severity levels




inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1

if K.image_data_format() == "channels_first":
    inputShape = (DEPTH, HEIGHT, WIDTH)
    chanDim = 1


# Initialize optimizer
opt = Adam(lr=LR, decay=LR / EPOCHS)

# Compile model
# model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

#  Compile the model with categorical cross-entropy loss and Adam optimizer
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Print model summary
model.summary()


# Train model
print("[INFO] Training network...")
# history = model.fit_generator(augment.flow(x_train, y_train, batch_size=BATCH_SIZE),
#                               validation_data=(x_test, y_test),
#                               steps_per_epoch=len(x_train) // BATCH_SIZE,
#                               epochs=EPOCHS, 
#                               verbose=1)

checkpoint = ModelCheckpoint('ctscan.hdf5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
# history = model.fit(x_train,y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data = (x_test, y_test),callbacks=[checkpoint])
# Train the model
history = model.fit(x_train, y_train_encoded, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test, y_test_encoded), callbacks=[checkpoint])


# Dump pickle file of the model
# model.save("ctscan1.hdf5")
print("[INFO] Saving model...")
# pickle.dump(model,open('ctscan.pkl', 'wb'))


[INFO] Splitting data to train and test...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flat

In [3]:
# li=['glioma','meningioma','notumor','pituitary']
# li=['adenocarcinoma','Normal']
li=['mri','non_mri']
import tensorflow
from keras.preprocessing import image
modelfile="ctscan.hdf5"
model = tensorflow.keras.models.load_model(modelfile)
from os import listdir
test_dir="C:\\Users\\cnet\\Downloads\\For_MRI\\testing\\"
dest_dir="D:\\datasets\\brain1\\test_images\\pituitary\\"
files = listdir(test_dir)
import shutil
for f in files:
    filename=test_dir+f
    dest=dest_dir+f
    new_img = image.load_img(filename, target_size=(224, 224))
    img = image.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    pred = model.predict(img)

    print(filename)
    d = pred.flatten()
#     print(d)
    j = d.max()
#     print(j)
    for index, item in enumerate(d):
        if item == j:
            class_name = li[index]
            break
    print(class_name)
    
#     elif class_name=='large_cell_carcinoma_left.hilum':
#         dest=dest_dir+"large.cell.carcinoma\\"+f
# #         ok_adenocarcinoma\\
# #         shutil.copy(filename, dest)
#     elif class_name=='normal':
#         dest=dest_dir+"normal\\"+f
# #         ok_adenocarcinoma\\
# #         shutil.copy(filename, dest)
#     elif class_name=='squamous_cell_carcinoma_left_hilum':
#         dest=dest_dir+"squamous.cell.carcinoma\\"+f
# #         ok_adenocarcinoma\\
# #         shutil.copy(filename, dest)

C:\Users\cnet\Downloads\For_MRI\testing\1.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\2.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\3-male.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\4-female.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\5-male.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\6-female.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\7-female.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\8-male.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\9-male.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-1.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-2.png
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-3.png
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-4.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-5.jpg
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c-6.png
non_mri
C:\Users\cnet\Downloads\For_MRI\testing\c10-1.jpg
mri
C:\Users\cnet\Downloads\For_MRI\testing\c12.1.jpg
mri
C:\Users\cnet\Downloads\For_M

In [23]:
import tensorflow
from keras.preprocessing import image
modelfile="ctscan.hdf5"
model = tensorflow.keras.models.load_model(modelfile)
def predict(fname):
    li=['Brain_Tumor','Healthy']
    filename=fname
    dest=dest_dir+f
    new_img = image.load_img(filename, target_size=(224, 224))
    img = image.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    pred = model.predict(img)
    d = pred.flatten()
    j = d.max()
    for index, item in enumerate(d):
        if item == j:
            class_name = li[index]
    print(class_name)

    
predict('D:\\datasets\\brain1\\Testing\\pituitary\\N3.jpg')

Healthy
